Title
===

Mert Sami

Myrthe Peeters

# Abstract

# Research Question

Are female doctors in Houston College of Medicine being discriminated against in giving promotions and setting salaries compared to their male counterparts in 1994 and 1995?

# Data

# Motivation 

# Method

# Results

# Conclusion